In [1]:
import os
import arrow
import dash
import chart_studio
import requests
from flask import Flask, json
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
import functools
from dash.dependencies import Input, Output
from dotenv import load_dotenv
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import plotly.express as px


/home/mousa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  
/home/mousa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  if __name__ == '__main__':


In [2]:
dotenv_path = os.path.join(os.path.dirname('example'), ".env")
load_dotenv(dotenv_path)

True

In [3]:
def create_dataframe(d):
    features = d["features"]
    properties = [x["properties"] for x in features]
    geometries = [x["geometry"] for x in features]
    coordinates = [x["coordinates"] for x in geometries]
    times = [convert_timestamp(x["time"]) for x in properties]
    dd = {
        "Place": [x["place"] for x in properties],
        "Magnitude": [x["mag"] for x in properties],
        "Time": times,
        "Detail": [x["detail"] for x in properties],
        "Longitude": [x[0] for x in coordinates],
        "Latitude": [x[1] for x in coordinates],
        "Depth": [x[2] for x in coordinates],
    }
    # html text to display when hovering
    texts = list()
    for i in range(len(properties)):
        text = "{}<br>{}<br>Magnitude: {}<br>Depth: {} km".format(
            dd["Time"][i], dd["Place"][i], dd["Magnitude"][i], dd["Depth"][i]
        )
        texts.append(text)
    dd.update({"Text": texts})
    return pd.DataFrame(dd)


In [4]:
def convert_timestamp(timestamp_ms):
    return arrow.get(timestamp_ms / 1000.0).format()


In [5]:
def create_metadata(d):
    dd = {
        "title": d["metadata"]["title"],
        "api": d["metadata"]["api"],
    }
    return dd

In [6]:
usgs = "http://earthquake.usgs.gov/earthquakes/"
geoJsonFeed = "feed/v1.0/summary/4.5_month.geojson"
url = "{}{}".format(usgs, geoJsonFeed)
req = requests.get(url)
data = json.loads(req.text)

In [7]:
dataframe = create_dataframe(data)
metadata = create_metadata(data)
print(dataframe.head())

                              Place  Magnitude                       Time  \
0           23 km NW of Pullo, Peru        5.2  2021-10-29 19:55:26+00:00   
1           Kermadec Islands region        5.7  2021-10-29 19:42:31+00:00   
2  southeast of the Loyalty Islands        4.7  2021-10-29 18:55:25+00:00   
3         south of the Fiji Islands        4.6  2021-10-29 15:23:23+00:00   
4   164 km N of Pangnirtung, Canada        4.7  2021-10-29 13:48:57+00:00   

                                              Detail  Longitude  Latitude  \
0  https://earthquake.usgs.gov/earthquakes/feed/v...   -74.0054  -15.0824   
1  https://earthquake.usgs.gov/earthquakes/feed/v...  -176.3393  -29.3901   
2  https://earthquake.usgs.gov/earthquakes/feed/v...   171.6169  -22.7973   
3  https://earthquake.usgs.gov/earthquakes/feed/v...  -179.6933  -23.8315   
4  https://earthquake.usgs.gov/earthquakes/feed/v...   -65.9352   67.6180   

    Depth                                               Text  
0   72.70  

In [8]:
# http://colorbrewer2.org/#type=sequential&scheme=YlOrRd&n=5
colorscale_magnitude = [
    [0, "#ffffb2"],
    [0.25, "#fecc5c"],
    [0.5, "#fd8d3c"],
    [0.75, "#f03b20"],
    [1, "#bd0026"],
]

# http://colorbrewer2.org/#type=sequential&scheme=Greys&n=3
colorscale_depth = [
    [0, 'rgb(240,240,240)'],
    [0.5, 'rgb(189,189,189)'],
    [1, 'rgb(99,99,99)']
]

theme = {
    "fontFamily": "Raleway",
    "backgroundColor": "#787878",
}





In [9]:
def create_td(series, col):
    val = series[col]
    if col == "Detail":
        td = html.Td(html.A(children="GeoJSON", href="{}".format(val), target="_blank"))
    else:
        td = html.Td(val,style={'color':'black','border-color':'black'})
    return td

In [10]:
def create_table(df):
    columns = ["Magnitude", "Latitude", "Longitude", "Time", "Place", "Detail"]
    num_rows = data["metadata"]["count"]
    thead = html.Thead(html.Tr([html.Th(col) for col in columns]),style={'color':'black'})
    table_rows = list()
    for i in range(num_rows):
        tr = html.Tr(
            children=list(map(functools.partial(create_td, df.iloc[i]), columns))
        )
        table_rows.append(tr)
    tbody = html.Tbody(children=table_rows)
    table = html.Table(children=[thead, tbody], id="my-table")
    return table

In [11]:
def create_header(some_string):
    header_style = {
        "backgroundColor": theme["backgroundColor"],
        "padding": "1.5rem",
    }
    header = html.Header(html.H1(children=some_string, style=header_style))
    return header

In [12]:
def create_footer():
    p = html.P(
        children=[
            html.Span("Built By :"),
            html.Span('Mohamed Mousa',style={'color':'white','font-family':'Times New Roman'}),
            html.Span(" and "),
            html.Span('Belal Ashraf',style={'color':'white','font-family':'Times New Roman'})
        ],
    )

    span_style = {"verticalAlign": "top", "paddingLeft": "1rem"}

    


    li_right_first = {"lineStyleType": "none", "display": "inlineBlock"}
    li_right_others = {k: v for k, v in li_right_first.items()}
    li_right_others.update({"marginLeft": "10px"})
    

    div = html.Div([p])
    footer_style = {
        "fontSize": "2.2rem",
        "backgroundColor": theme["backgroundColor"],
        "padding": "2.5rem",
        "marginTop": "3rem",
    }
    footer = html.Footer(div, style=footer_style)
    return footer



In [13]:
def create_dropdowns():
    drop1 = dcc.Dropdown(
        options=[
            {"label": "Light", "value": "light"},
            {"label": "Dark", "value": "dark"},
            {"label": "Satellite", "value": "satellite"},
            
        ],
        value="dark",
        id="dropdown-map-style",
        className="three columns offset-by-one",
    )
    drop2 = dcc.Dropdown(
        options=[
            {"label": "World", "value": "world"},
            {"label": "Europe", "value": "europe"},
            {"label": "North America", "value": "north_america"},
            {"label": "South America", "value": "south_america"},
            {"label": "Africa", "value": "africa"},
            {"label": "Asia", "value": "asia"},
            {"label": "Oceania", "value": "oceania"},
        ],
        value="world",
        id="dropdown-region",
        className="three columns offset-by-four",
    )
    return [drop1, drop2]


In [14]:
def create_description():
    div = html.Div(
        children=[
            dcc.Markdown(
                """
            Use the table below to know more about the {} earthquakes that 
            exceeded magnitude 4.5 last month.

            ***
            """.format(
                    data["metadata"]["count"]
                ).replace(
                    "  ", ""
                )
            ),
        ],
    )
    return div

In [15]:
def create_content(df):
    # create empty figure. It will be updated when _update_graph is triggered
    px.set_mapbox_access_token('pk.eyJ1IjoiYW1yLWthc2VtLWI1NyIsImEiOiJja3VtZW52aHgwc2RmMnFtb2ljYm56aWFwIn0.PBQ3U_2nlEgoqtcnzIrMDw')
    # can get API key from  "https://chart-studio.plotly.com/feed/#/"  
    fig= px.scatter_mapbox(df, hover_name='Place', lat='Latitude',
                                        lon='Longitude',mapbox_style='satellite',text=df['Magnitude']

                            )
    graph = dcc.Graph(id="geo",figure=fig)
    content = html.Div(graph, id="content")
    return content

In [16]:
regions = {
    "world": {"lat": 0, "lon": 0, "zoom": 1},
    "europe": {"lat": 50, "lon": 0, "zoom": 3},
    "north_america": {"lat": 40, "lon": -100, "zoom": 2},
    "south_america": {"lat": -15, "lon": -60, "zoom": 2},
    "africa": {"lat": 0, "lon": 20, "zoom": 2},
    "asia": {"lat": 30, "lon": 100, "zoom": 2},
    "oceania": {"lat": -10, "lon": 130, "zoom": 2},
}


In [17]:
app_name = "World Earthquakes Live"

app = dash.Dash(name=app_name , title='Mousa&belal_World_Earthquakes_Live')

app.layout = html.Div(
    children=[
         create_header(app_name),
        html.Div(
            children=[
                 html.Div(create_dropdowns(), className="row"),
                 html.Div(create_content(dataframe), className="row"),
                 html.Div(create_description(), className="row"),
                 html.Div(create_table(dataframe), className="row"),
            ],
        ),
         html.Hr(),
         create_footer(),
    ],
    className="container",
    style={"fontFamily": theme["fontFamily"]},
)



In [18]:
@app.callback(
    output=Output("geo", "figure"),
    inputs=[Input("dropdown-map-style", "value"), Input("dropdown-region", "value")],
)

def _update_graph(map_style, region):
    dff = dataframe
    radius_multiplier = {"inner": 1.5, "outer": 3}
    mapbox_access_token='pk.eyJ1IjoiYmVsYWxsbCIsImEiOiJja3Y2aXJ2YTAxOHd5MnJsdXpnaW5wYXY1In0.1AvWAZxF8v624NlWEgEMLQ'

    layout = go.Layout(
        title=metadata["title"],
        autosize=True,
        hovermode="closest",
        height=750,
        font=dict(family=theme["fontFamily"]),
        margin=go.layout.Margin(l=0, r=0, t=45, b=10),
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=regions[region]["lat"],
                lon=regions[region]["lon"],
            ),
            pitch=0,
            zoom=regions[region]["zoom"],
            style=map_style,
        ),
    )

    # outer circles represent magnitude
    # inner circles represent depth
    data = (
        go.Scattermapbox(
            lat=dff["Latitude"],
            lon=dff["Longitude"],
            mode="markers",
            marker=go.scattermapbox.Marker(
                color=dff["Magnitude"],
                colorscale=colorscale_magnitude,
                opacity=1,
                # showscale=True,
                size=dff["Magnitude"] * radius_multiplier["outer"],
            ),
            text=dff["Text"],
            # hoverinfo='text',
            showlegend=True,
        ),
        go.Scattermapbox(
            lat=dff["Latitude"],
            lon=dff["Longitude"],
            mode="markers",
            marker=go.scattermapbox.Marker(
                color=dff["Depth"],
                colorscale=colorscale_depth,
                opacity=1,
                # showscale=True,
                size=dff["Magnitude"] * radius_multiplier["inner"],
            ),
            # hovering behavior is already handled by outer circles
            hoverinfo="skip",
            showlegend=True,
        ),
    )

    figure = go.Figure(data=data, layout=layout)
    return figure


In [20]:
app.run_server(port=8080, threaded=True)


Dash is running on http://127.0.0.1:8080/

 * Serving Flask app "World Earthquakes Live" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
